# 단어의 표현 (Word Representation)


기계는 문자를 그대로 인식할 수 없기때문에 숫자로 변환



# 1 원-핫 인코딩 (One-Hot Encoding)

## 1.1 직접 구현해보기

### "원숭이, 바나나, 사과" 로 원-핫 인코딩을 한다면

In [63]:
# 인코딩 대상 단어들을 담은 리스트
word_ls = ['원숭이','바나나','사과','사과']

In [65]:
word_dict = {}
for i, word in enumerate(set(word_ls)):
    word_dict[word] = i
word_dict

#이 작업을 한번에 해주는 defaultdict()

{'바나나': 0, '원숭이': 1, '사과': 2}

In [68]:
from collections import defaultdict
word_dict = defaultdict(lambda : len(word_dict))
for word in word_ls:
    word_dict[word]
word_dict

defaultdict(<function __main__.<lambda>()>, {'원숭이': 0, '바나나': 1, '사과': 2})

In [4]:
# 값을 집어 넣지 않으면 에러가 남
words = {"원숭이":0}
words['바나나']

KeyError: '바나나'

In [7]:
# default값으로 새로 생성
from collections import defaultdict
words_default = defaultdict(lambda : len(words_default))
words_default['원숭이'] = 0
words_default['바나나']
words_default

defaultdict(<function __main__.<lambda>()>, {'원숭이': 0, '바나나': 1})

In [38]:
from collections import defaultdict
import numpy as np 

def one_hot_encode(word_ls):
    word2id = defaultdict(lambda : len(word2id))
    
    for word in word_ls:
        word2id[word]
        
    print("word2id : ", word2id)
    unique_words = len(word2id)
    one_hot_vectors = np.zeros((len(word_ls), unique_words))
#     print("one_hot_vectors : ", one_hot_vectors)
    
    # 단어 자리에 1 넣기
    for i, word in enumerate(word_ls):
        index = word2id[word]
        one_hot_vectors[i, index] = 1
        
    return one_hot_vectors

In [34]:
one_hot_encode(word_ls)

word2id :  defaultdict(<function one_hot_encode.<locals>.<lambda> at 0x0000023AD720AEE8>, {'원숭이': 0, '바나나': 1, '사과': 2})
one_hot_vectors :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [43]:
# 단어 자리에 1넣기
one_hot_encode(word_ls)

word2id :  defaultdict(<function one_hot_encode.<locals>.<lambda> at 0x0000023AD721D798>, {'원숭이': 0, '바나나': 1, '사과': 2})


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

### "코끼리"라는 단어가 추가된다면?

In [44]:
word_ls = ['원숭이','바나나','사과','코끼리']

In [45]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

word2id :  defaultdict(<function one_hot_encode.<locals>.<lambda> at 0x0000023ADF22A678>, {'원숭이': 0, '바나나': 1, '사과': 2, '코끼리': 3})


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## 1.3 sklearn 활용


함수명 | 설명
--|--
fit(X[, y])	| Fit OneHotEncoder to X.
fit_transform(X[, y])	| Fit OneHotEncoder to X, then transform X.
inverse_transform(X)	| Convert the back data to the original representation.
transform(X)	| Transform X using one-hot encoding.

In [54]:
# sklearn을 활용한 one-hot encoding
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = np.array(word_ls)
print(values)

label_enc = LabelEncoder()
int_enc = label_enc.fit_transform(values)
print(int_enc)
print(int_enc.shape)

onehot_enc = OneHotEncoder()
int_enc = int_enc.reshape(-1, 1)
print(int_enc)
print(int_enc.shape)

onehot_enc = onehot_enc.fit_transform(int_enc)
print(onehot_enc)
print(onehot_enc.toarray())

['원숭이' '바나나' '사과' '코끼리']
[2 0 1 3]
(4,)
[[2]
 [0]
 [1]
 [3]]
(4, 1)
  (0, 2)	1.0
  (1, 0)	1.0
  (2, 1)	1.0
  (3, 3)	1.0
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [55]:
onehot_enc[0, :]

<1x4 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [56]:
# 가장 큰 수의 인덱스
np.argmax(onehot_enc[0, :])

2

In [61]:
label_enc.inverse_transform([np.argmax(onehot_enc[0, :])])

array(['원숭이'], dtype='<U3')



---



# 2 밀집 벡터 (Dense Vector)

In [72]:
word_embedding_dic = {
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

## 2-1 유사도 계산

### 2.1.1 유클리디언 거리(Euclidean distance)
두 벡터사이의 직선 거리. 피타고라스 정리를 생각하면 이해하기 쉬움

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Euclidean_distance_2d.svg/220px-Euclidean_distance_2d.svg.png"  width="200"/>

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0" width="350"/>

https://en.wikipedia.org/wiki/Euclidean_distance

In [71]:
import numpy as np
def euclidean_dist(x,y):   
    x = np.array(x) # [1.0, 0.5]
    y = np.array(y) # [0.9, 1.2]
    return np.sqrt(np.sum((x - y)**2))

# 사과와 바나나의 유클리디안 유사도
euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나'])

0.7071067811865475

<img src="https://www.oreilly.com/library/view/statistics-for-machine/9781788295758/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png" width="300"/>

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d" width='350'/>

https://en.wikipedia.org/wiki/Cosine_similarity

### 2.1.2 코사인 유사도(Cosine Similarity) 

*  두 벡터간의 유사도를 측정하는 방법 중 하나
*  두 벡터 사이의 코사인을 측정
*  0도 = 1, 90도 = 0, 180도 = -1   ==> 1에 가까울수록 유사도가 높음




In [77]:
def cosine_similarity(x, y):
    nom = np.dot(x, y)
    dem = np.linalg.norm(x) * np.linalg.norm(y)
    return nom/dem

In [74]:
a = np.array(word_embedding_dic['사과'])
b = np.array(word_embedding_dic['바나나'])
np.dot(a, b)

1.5

numpy의 linalg 서브 패키지의 norm 명령으로 벡터의 길이를 계산할 수 있다. 위에서 예로 든 2차원 벡터  𝑎=[1,2] 의 길이는  √5≈2.236 이다.

In [76]:
# 크기
a = np.array([1,2])
np.linalg.norm(a)

2.23606797749979

In [78]:
# 사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))

0.8944271909999159
0.7071067811865475


In [81]:
# 사과와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['원숭이']))

0.7071067811865475
1.118033988749895


In [80]:
# 바나나와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))

0.9486832980505138
0.5


### 2.1.3 자카드 유사도(Jaccard index)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Intersection_of_sets_A_and_B.svg/200px-Intersection_of_sets_A_and_B.svg.png" />

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/eaef5aa86949f49e7dc6b9c8c3dd8b233332c9e7" />

https://en.wikipedia.org/wiki/Jaccard_index

In [84]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

# 토큰화
token_1 = s1.split()
token_2 = s2.split()

print(token_1)
print(token_2)

#합집합
union_sent = set(token_1).union(set(token_2))
print(union_sent)

#교집합
inter_sent = set(token_1).intersection(set(token_2))
print(inter_sent)

print(len(inter_sent)/len(union_sent))

['대부분', '원숭이는', '바나나를', '좋아합니다.']
['코주부', '원숭이는', '바나나를', '싫어합니다.']
{'원숭이는', '코주부', '바나나를', '좋아합니다.', '싫어합니다.', '대부분'}
{'원숭이는', '바나나를'}
0.3333333333333333
